This notebook contains modifications made to original datasets. It includes changing column names, removing junk lines, etc.

In [2]:
import pandas as pd

### RODS

In [3]:
df = pd.read_excel('/Users/itto/Downloads/RODS_2017/Misc/Origin-destination matrix by station-zone-time of day 2017.xls',
                     skiprows=2)

In [4]:
df.head()

,From,Unnamed: 1,To,Unnamed: 3,Early,AM peak,Midday,PM Peak,Evening,Late,Weekday
0,NaN,NaN,NaN,NaN,- 7am,7am-10am,10am-4pm,4pm-7pm,7pm-10pm,10pm+,Total
1,500.0,Acton Town,505.0,Alperton,0,44,0,93,0,0,137
2,500.0,Acton Town,506.0,Amersham,0,0,0,20,0,0,20
3,500.0,Acton Town,507.0,Angel,1,13,5,0,3,0,22
4,500.0,Acton Town,508.0,Archway,0,0,0,0,0,4,4


In [5]:
df.drop(0, inplace=True)
df.drop(['From','To'],axis=1, inplace=True)
df.columns = ['origin','destination','early','morning','midday','afternoon','evening','night','total']
df.origin = df.origin.str.replace('\s\s+','')
df.destination = df.destination.str.replace('\s\s+','')

In [6]:
df.head()

,origin,destination,early,morning,midday,afternoon,evening,night,total
1,Acton Town,Alperton,0,44,0,93,0,0,137
2,Acton Town,Amersham,0,0,0,20,0,0,20
3,Acton Town,Angel,1,13,5,0,3,0,22
4,Acton Town,Archway,0,0,0,0,0,4,4
5,Acton Town,Arsenal,0,0,12,17,0,0,29


In [9]:
# Rename Hammersmiths
df.loc[df.origin.str.contains('Hammersmith (Dis)', regex=False), 'origin'] = "Hammersmith 1"
df.loc[df.origin.str.contains('Hammersmith (H&C)', regex=False), 'origin'] = "Hammersmith 2"
df.loc[df.destination.str.contains('Hammersmith (Dis)', regex=False), 'destination'] = "Hammersmith 1"
df.loc[df.destination.str.contains('Hammersmith (H&C)', regex=False), 'destination'] = "Hammersmith 2"
# Combine the Shepherd's bush
df.loc[df.origin.str.contains('Bush (Cen)', regex=False), 'origin'] = "Shepherd's Bush 1"
df.loc[df.origin.str.contains('Bush Market', regex=False), 'origin'] = "Shepherd's Bush 1"
df.loc[df.destination.str.contains('Bush (Cen)', regex=False), 'destination'] = "Shepherd's Bush 1"
df.loc[df.destination.str.contains('Bush Market', regex=False), 'destination'] = "Shepherd's Bush 1"

In [77]:
# Add the counts again now that we combined the stations
df = df.groupby(['origin','destination']).sum()

In [78]:
df.reset_index(inplace=True)

In [79]:
df.to_csv('../data/london/counts/rods.csv',index=False)

### Oyster

In [23]:
df = pd.read_csv('../data/london/counts/Nov09JnyExport.csv')

In [24]:
df.head()

,downo,daytype,SubSystem,StartStn,EndStation,EntTime,EntTimeHHMM,ExTime,EXTimeHHMM,ZVPPT,JNYTYP,DailyCapping,FFare,DFare,RouteID,FinalProduct
0,3,Tue,LUL,Unstarted,Kings Cross M,0,00:00,633,10:33,Z0104,TKT,N,0,0,XX,LUL Travelcard-7 Day
1,4,Wed,LUL,Unstarted,Sudbury Hill,0,00:00,447,07:27,Z0110,TKT,N,0,0,XX,Freedom Pass (Elderly)
2,3,Tue,NR,Unstarted,Richmond,0,00:00,966,16:06,Z0304,TKT,N,0,0,XX,LUL Travelcard-7 Day
3,4,Wed,NR,Unstarted,Romford,0,00:00,657,10:57,Z0110,TKT,N,0,0,XX,Freedom Pass (Elderly)
4,6,Fri,NR,Unstarted,Norwood Junction SR,0,00:00,450,07:30,Z0104,TKT,N,0,0,XX,LUL Travelcard-7 Day


In [26]:
df.drop(['ZVPPT','JNYTYP','DailyCapping','FFare','DFare','RouteID','FinalProduct'], axis=1, inplace=True)

In [27]:
df.StartStn = df.StartStn.str.lower()
df.EndStation = df.EndStation.str.lower()

In [28]:
df.StartStn = df.StartStn.str.replace(' rd',' road')
df.EndStation = df.EndStation.str.replace(' rd',' road')

In [29]:
df.head()

,downo,daytype,SubSystem,StartStn,EndStation,EntTime,EntTimeHHMM,ExTime,EXTimeHHMM
0,3,Tue,LUL,unstarted,kings cross m,0,00:00,633,10:33
1,4,Wed,LUL,unstarted,sudbury hill,0,00:00,447,07:27
2,3,Tue,NR,unstarted,richmond,0,00:00,966,16:06
3,4,Wed,NR,unstarted,romford,0,00:00,657,10:57
4,6,Fri,NR,unstarted,norwood junction sr,0,00:00,450,07:30


In [30]:
df.to_csv('../data/london/counts/oyster.csv',index=False)